In [2]:
import os
import torch
import pandas as pd
import numpy as np
import model_AD_1 as AD
import pickle

%load_ext autoreload
%autoreload 2


In [17]:
x_pos = np.load('./../generated_data_v1/us_import1/stage_2/train_x_pos.npy')
x_neg = np.load('./../generated_data_v1/us_import1/stage_2/train_x_neg.npy')

In [23]:
x_neg = x_neg.reshape([x_pos.shape[0], -1 , x_pos.shape[1]])

In [18]:
with open('./../generated_data_v1/us_import1/domain_dims.pkl','rb')  as fh:
    domain_dims= pickle.load(fh)
    

In [19]:
total_entity_count = sum(domain_dims.values())

In [26]:
model = AD.AD_model_container(total_entity_count, emb_dim=16)

In [ ]:
model.train_model(x_pos,x_neg, batch_size=256, epochs=20)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 0  batch 0 Loss 1.073007
Epoch 0  batch 100 Loss 1.004386
Epoch 0  batch 200 Loss 0.948013
Epoch 0  batch 300 Loss 0.972287
Epoch 0  batch 400 Loss 0.915959
Epoch 0  batch 500 Loss 0.909681


  5%|▌         | 1/20 [01:24<26:41, 84.30s/it]

Epoch 1  batch 0 Loss 0.875377
Epoch 1  batch 100 Loss 0.831254
Epoch 1  batch 200 Loss 0.757903
Epoch 1  batch 300 Loss 0.819204
Epoch 1  batch 400 Loss 0.726067
Epoch 1  batch 500 Loss 0.761799


 10%|█         | 2/20 [02:48<25:16, 84.23s/it]

Epoch 2  batch 0 Loss 0.696423
Epoch 2  batch 100 Loss 0.737410
Epoch 2  batch 200 Loss 0.708994
Epoch 2  batch 300 Loss 0.629264
Epoch 2  batch 400 Loss 0.653498
Epoch 2  batch 500 Loss 0.662399


 15%|█▌        | 3/20 [04:11<23:45, 83.87s/it]

Epoch 3  batch 0 Loss 0.690247
Epoch 3  batch 100 Loss 0.630665
Epoch 3  batch 200 Loss 0.564347
Epoch 3  batch 300 Loss 0.585909
Epoch 3  batch 400 Loss 0.566188
Epoch 3  batch 500 Loss 0.584829


 20%|██        | 4/20 [05:32<22:08, 83.04s/it]

Epoch 4  batch 0 Loss 0.568169
Epoch 4  batch 100 Loss 0.548959
